In [1]:
sealed trait Result[+A]
final case class Success[A](result: A) extends Result[A]
final case class Failure(reason: String) extends Result[Nothing]

defined trait Result
defined class Success
defined class Failure

In [2]:
sealed trait LinkedList[+T] {
    def apply(n: Int): Result[T] = this match {
        case End => Failure("Out of bounds")
        case Cons(head: T, tail) =>
            if (n == 0) Success(head) else apply(n - 1)
    }
    
    def foldr[B](acc: B, f: (T, B) => B): B = this match {
        case End => acc
        case Cons(head, tail) => f(head, tail.foldr(acc, f))
    }
    
    def extend[TT >: T](l: LinkedList[TT]): LinkedList[TT] = foldr[LinkedList[TT]](l, (head, tail) => Cons(head, tail))
    
    def map[B](f: T => B): LinkedList[B] = foldr[LinkedList[B]](End, (head, tail) => Cons(f(head), tail))
    
    def flatMap[B](f: T => LinkedList[B]): LinkedList[B] = foldr[LinkedList[B]](End, (head, tail) => f(head).extend(tail))
}
final case object End extends LinkedList[Nothing]
final case class Cons[T](head: T, tail: LinkedList[T]) extends LinkedList[T]

val test_list = Cons(1, Cons(5, Cons(4, End)))

defined trait LinkedList
defined object End
defined class Cons
test_list: Cons[Int] = Cons(1, Cons(5, Cons(4, End)))

In [3]:
test_list(2)
test_list(3)

res2_0: Result[Int] = Success(1)
res2_1: Result[Int] = Success(1)

In [4]:
test_list.foldr[Int](0, (_, v) => 1 + v)
test_list.map(_ * 2)
test_list.flatMap(v => if (v % 2 == 0) Cons(v * 200, Cons(v * 2, End)) else Cons(v, End))

res3_0: Int = 3
res3_1: LinkedList[Int] = Cons(2, Cons(10, Cons(8, End)))
res3_2: LinkedList[Int] = Cons(1, Cons(5, Cons(800, Cons(8, End))))

In [5]:
sealed trait Tree[+T] {
    def foldr[B](acc: B)(f: (T, B, B) => B): B
}
final case class Node[T](value: T, left: Tree[T], right: Tree[T]) extends Tree[T] {
    def foldr[B](acc: B)(f: (T, B, B) => B): B =
        f(value, left.foldr(acc)(f), right.foldr(acc)(f))
}
final case object End extends Tree[Nothing] {
    def foldr[B](acc: B)(f: (Nothing, B, B) => B): B = acc
}

val test_tree = Node(5, Node(4, End, End), End)

defined trait Tree
defined class Node
defined object End
test_tree: Node[Int] = Node(5, Node(4, End, End), End)

In [6]:
test_tree.foldr(0)((v, l, r) => v + l + r)

res5: Int = 9

In [7]:
final case class Pair[A, B](one: A, two: B)
var test_pair = Pair[Int, String](1, "TMNT")
test_pair.two

defined class Pair
test_pair: Pair[Int, String] = Pair(1, "TMNT")
res6_2: String = "TMNT"

In [8]:
sealed trait Sum[+A, +B]
final case class Failure[A, Nothing](value: A) extends Sum[A, Nothing]
final case class Success[B](value: B) extends Sum[Nothing, B]

defined trait Sum
defined class Failure
defined class Success

In [9]:
sealed trait Maybe[+A] {
    def fold[B](full: A => B, empty: B): B = this match {
        case Full(a) => full(a)
        case Empty => empty
    }
    
    def flatMap[B](f: A => Maybe[B]): Maybe[B] = this.fold[Maybe[B]](f, Empty)
    
    def map[B](f: A => B): Maybe[B] = this.flatMap[B](v => Full(f(v)))
}
final case class Full[A](value: A) extends Maybe[A]
final case object Empty extends Maybe[Nothing]

defined trait Maybe
defined class Full
defined object Empty

In [10]:
val list = List(1, 2, 3)
list.flatMap(v => List(v, -v))

list: List[Int] = List(1, 2, 3)
res9_1: List[Int] = List(1, -1, 2, -2, 3, -3)

In [11]:
val list = List(Full(3), Full(2), Full(1))
list.map(mv => {
    mv.flatMap(v => {
        if (v % 2 == 0) Full(v) else Empty
    })
})

list: List[Full[Int]] = List(Full(3), Full(2), Full(1))
res10_1: List[Maybe[Int]] = List(Empty, Full(2), Empty)

In [12]:
sealed trait Sum[+A, +B] {
    def map[C](f: B => C): Sum[A, C] = this match {
        case Failure(v) => Failure(v)
        case Success(v) => Success(f(v))
    }
    
    def flatMap[AA >: A, C](f: B => Sum[AA, C]): Sum[AA, C] = this match {
        case Failure(v) => Failure(v)
        case Success(v) => f(v)
    }
}
final case class Failure[A](value: A) extends Sum[A, Nothing]
final case class Success[B](value: B) extends Sum[Nothing, B]

defined trait Sum
defined class Failure
defined class Success

In [13]:
sealed trait Expression {
    def eval: Sum[String, Double]
}

final case class Addition(left: Expression, right: Expression) extends Expression {
    def eval: Sum[String, Double] =
        left.eval.flatMap {
            lv => right.eval.map {
                rv => lv + rv
            }
        }
}

final case class Subtraction(left: Expression, right: Expression) extends Expression {
    def eval: Sum[String, Double] =
        left.eval.flatMap {
            lv => right.eval.map {
                rv => lv - rv
            }
        }
}

final case class Number(value: Double) extends Expression {
    def eval: Sum[String, Double] = Success(value)
}

Addition(Number(1), Number(2)).eval
Subtraction(Number(1), Number(2)).eval

defined trait Expression
defined class Addition
defined class Subtraction
defined class Number
res12_4: Sum[String, Double] = Success(3.0)
res12_5: Sum[String, Double] = Success(-1.0)